# Vorverarbeitung der Rohdaten für die Clusteranalyse

In [ ]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
d_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv(r"Dateipfad\Rohdaten_Ladesäulen.csv",sep=';', encoding='latin-1', decimal= ',', parse_dates=['Gestartet', 'Beendet'], date_parser=d_parser)

In [ ]:
df.columns

In [ ]:
df = df[['Ladepunkt','Gestartet', 'Beendet','Standzeit','Ladezeit','Verbrauch (kWh)','kW_max', 'kW_mean', 'kW_min']]

In [ ]:
## Enfternung der leeren Einträge

df = df.dropna()

In [ ]:
## Anpassung des Datenformats

df["Standzeit"] = df["Standzeit"].astype(float).astype(int)  
df["Verbrauch (kWh)"] = df["Verbrauch (kWh)"].astype(float) 
df["kW_min"] = df["kW_min"].astype(float) 
df["kW_max"] = df["kW_max"].astype(float) 
df["kW_mean"] = df["kW_mean"].astype(float) 
df["Ladezeit"] = df["Ladezeit"].astype(float).astype(int) 

In [ ]:
## Entfernung der Ladevorgänge ohne Ladekurven  

df = df[df["kW_min"] != df["kW_max"]]

In [ ]:
## Untersuchung der Merkmale mit statistischen Größen 

df.describe()

In [ ]:
df["Monat"] = df["Gestartet"].dt.month_name()
df["Wochentag"] = df["Gestartet"].dt.day_name()
df["Gestartet_hour"] = df["Gestartet"].dt.hour
df["Gestartet_minute"] = df["Gestartet"].dt.minute
df["Gestartet_sekunde"] = df["Gestartet"].dt.second
df["Beendet_hour"] = df["Beendet"].dt.hour 
df["Beendet_minute"] = df["Beendet"].dt.minute
df["Beendet_sekunde"] = df["Beendet"].dt.second

In [ ]:
df["Leerlaufzeit"] = df["Standzeit"] - df["Ladezeit"]

In [ ]:
df['Gestartet_Stunden'] = df["Gestartet_hour"] + df["Gestartet_minute"]/60 + df["Gestartet_sekunde"]/3600
df['Beendet_Stunden'] = df["Beendet_hour"]  + df["Beendet_minute"]/60 + df["Beendet_sekunde"]/3600

In [ ]:
df['Standzeit'] = df['Standzeit']/3600
df['Ladezeit'] = df['Ladezeit']/3600
df['Leerlaufzeit'] = df['Leerlaufzeit']/3600

In [ ]:
df = df.rename(columns={'Gestartet_Stunden': 'Ankunftszeit'})
df = df.rename(columns={'Beendet_Stunden': 'Abfahrtszeit'})
df = df.rename(columns={'Leerlauf': 'Leerlaufzeit'})
df = df.rename(columns={'kW_max': 'Leistung_max (kW)'})

In [ ]:
df = df[['Ladepunkt','Ankunftszeit','Abfahrtszeit','Standzeit','Ladezeit'
         ,'Leerlaufzeit','Leistung_max (kW)','Verbrauch (kWh)','Monat','Wochentag',]]

In [ ]:
df.describe()

In [ ]:
df = df[df["Standzeit"] > 0]
df = df[df["Ladezeit"] > 0]
df = df[df["Leerlaufzeit"] >= 0]
df = df[df["Verbrauch (kWh)"] > 0]
df = df[df["Leistung_max (kW)"] > 0]
df = df[df["Standzeit"] > 0.1]

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df = df[['Ladepunkt','Ankunftszeit','Abfahrtszeit','Standzeit','Ladezeit',
         'Leerlaufzeit','Leistung_max (kW)','Verbrauch (kWh)','Monat','Wochentag',]]

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
## Untersuchung der Korrelation 

plt.figure(figsize = (16,8))
korrelation = df.corr()
sns.heatmap(korrelation, cmap= "GnBu", annot = True)

# plt.savefig('Korrelationsmatrix.png', bbox_inches='tight')

In [ ]:
df["Ladeanteil"] = (df["Ladezeit"]/df["Standzeit"])*100

In [ ]:
def Musterplot(ax, title, xlabel, ylabel, xticks, yticks): 
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)

    ax.xaxis.set_tick_params(top='on', direction='in', width=1)
    ax.yaxis.set_tick_params(right='on', direction='in', width=1)
    
    ax.set_title(title,fontweight='bold',fontsize=14)
    
    ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
    ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)
    
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

In [ ]:
## Histogramm von Ankunfts- und Abfahrtszeit 

fig, ax = plt.subplots(1,1,figsize=(8,4))

xlabel = "Zeit in h"
ylabel = "Anzahl der Ladevorgänge"
title = "Ladevorgänge in Abhängigkeit der Ankunfts- und Abfahrtszeit"
yticks = np.arange(0, 200, step=20)
xticks = np.arange(0, 25, step=1)
plt.grid()
ax.set_axisbelow(True)
ax.hist(df['Ankunftszeit'],bins = 40, alpha=0.5, edgecolor = "black", label='Ankunftszeit')
ax.hist(df['Abfahrtszeit'],bins = 40, alpha=0.5, edgecolor = "black", label='Abfahrzeit')
plt.legend(loc='upper right')
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

# fig.savefig("hist_AN_AB", dpi=600)

In [ ]:
## Streudiagramm von Ankunfts- und Abfahrzeit

fig, ax = plt.subplots(1,1,figsize=(7,5))

xlabel = "Ankunftszeit in h"
ylabel = "Abfahrtszeit in h"
title = "Zusammenhang zwischen Ankunfts- und Abfahrtszeit"
yticks = np.arange(0, 25, step=1)
xticks = np.arange(0, 25, step=1)
plt.grid()
ax.set_axisbelow(True)
ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'])
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

# fig.savefig("Scatter_AN_AB")

In [ ]:
## Entfernung der Ausreißer bei Ankunfts- und Abfahrtszeit 

df = df[df["Ankunftszeit"] > 5]
df = df[df["Abfahrtszeit"] > 5]

In [ ]:
## Streudiagramm von max. Leistung und Gesamtverbrauch 

fig, ax = plt.subplots(1,1,figsize=(7,5))

xlabel = "max. Leistung in kW"
ylabel = "Gesamtverbrauch in kWh"
title = "Zusammenhang zwischen Verbrauch und max. Leistung"
yticks = np.arange(0, 80, step=5)
xticks = np.arange(0, 25, step=1)
plt.grid()
ax.set_axisbelow(True)
ax.scatter(df['Leistung_max (kW)'],df['Verbrauch (kWh)'])
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

# fig.savefig("Scatter_kW_kWh")

In [ ]:
## Histogramm von max. Leistung 

fig, ax = plt.subplots(1,1,figsize=(8,4))

xlabel = "max. Leistung in kW"
ylabel = "Anzahl der Ladevorgänge"
title = "Ladevorgänge in Abhängigkeit der max. Leistung"
xticks = np.arange(0, 24, step=1)
yticks = np.arange(0, 480, step=40)
plt.grid()
ax.set_axisbelow(True)
ax.hist(df['Leistung_max (kW)'], bins = 25, edgecolor = "black",rwidth=0.8)
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

# fig.savefig("hist_Leistung")

In [ ]:
## Histogramm von Ladeanteil

fig, ax = plt.subplots(1,1,figsize=(8,4))

xlabel = "Ladeanteil in %"
ylabel = "Anzahl der Ladevorgänge"
title = "Ladevorgänge in Abhängigkeit des Ladeanteils"
xticks = np.arange(0, 110, step=10)
yticks = np.arange(0, 400, step=40)
plt.grid()
ax.set_axisbelow(True)
ax.hist(df['Ladeanteil'], bins = 25, edgecolor = "black",rwidth=.8,align='right')
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

# fig.savefig("hist_Ladeanteil")

In [ ]:
## Histogramm von Wochentagen 

df['Wochentag'] = pd.Categorical(df['Wochentag'], ['Monday','Tuesday','Wednesday','Thursday', 'Friday','Saturday','Sunday'])

fig, ax = plt.subplots(figsize=(8,4))

ax = sns.histplot(data = df, 
            x = 'Wochentag',
            shrink = 0.8, 
            discrete = True,
            )
plt.grid()

ax.set_axisbelow(True)
ax.set_title("Ladevorgänge in Abhänigkeit der Wochentage")
ax.set(xlabel = 'Anzahl der Ladevorgänge', ylabel='Wochentag')
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.xaxis.set_tick_params(top='on', direction='in', width=1)
ax.yaxis.set_tick_params(right='on', direction='in', width=1)
ax.set_title(title,fontweight='bold',fontsize=14)
ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)

# fig.savefig("hist_Wochentag")


In [ ]:
df.to_csv("Datensatz_Clusteranalyse.csv", sep = ";",index=False)